In [1]:
!pip install -U efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.6 MB/s eta 0:00:00


In [1]:
data_dir = 'train'

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Flatten, BatchNormalization, Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from efficientnet.keras import EfficientNetB0
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to load and split your dataset
def load_and_split_data(data_dir, test_size=0.2, random_state=42):
    data_gen = ImageDataGenerator(rescale=1./255, validation_split=test_size)
    train_set = data_gen.flow_from_directory(
        data_dir,
        target_size=(224, 224),  # Revert back to 224x224
        batch_size=32,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    test_set = data_gen.flow_from_directory(
        data_dir,
        target_size=(224, 224),  # Revert back to 224x224
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    return train_set, test_set

training_set, test_set = load_and_split_data(data_dir)



Found 517 images belonging to 3 classes.
Found 129 images belonging to 3 classes.


In [3]:
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
    pooling=None,
    classes=3  
)

In [4]:
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Output layer for binary classification
model.add(Dense(3, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b0 (Functiona  (None, 7, 7, 1280)        4049564   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 7, 7, 1280)        0         
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 batch_normalization (Batch  (None, 62720)             250880    
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 32)                2007072   
                                                                 
 batch_normalization_1 (Bat  (None, 32)                1

In [6]:
METRICS = [
    BinaryAccuracy(name='accuracy'),
    Precision(name='precision'),
    Recall(name='recall'),
                                
    AUC(name='auc'),
]

# Define callbacks
lrd = ReduceLROnPlateau(monitor='val_loss', patience=20, verbose=1, factor=0.50, min_lr=1e-10)
mcp = ModelCheckpoint('model.h5')
es = EarlyStopping(verbose=1, patience=20)

In [7]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=METRICS)

In [8]:
# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    verbose=1,
    callbacks=[lrd, mcp, es]
)

Epoch 1/10
17/17 [==============================] - ETA: 0s - loss: 0.8670 - accuracy: 0.7660 - precision: 0.6799 - recall: 0.5629 - auc: 0.8042

/Users/realaryansharma/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


17/17 [==============================] - 8s 373ms/step - loss: 0.8670 - accuracy: 0.7660 - precision: 0.6799 - recall: 0.5629 - auc: 0.8042 - val_loss: 2.1568 - val_accuracy: 0.7623 - val_precision: 0.6434 - val_recall: 0.6434 - val_auc: 0.7947 - lr: 0.0010
Epoch 2/10
17/17 [==============================] - 6s 347ms/step - loss: 0.3514 - accuracy: 0.9407 - precision: 0.9493 - recall: 0.8685 - auc: 0.9807 - val_loss: 1.0770 - val_accuracy: 0.7881 - val_precision: 0.6880 - val_recall: 0.6667 - val_auc: 0.8556 - lr: 0.0010
Epoch 3/10
17/17 [==============================] - 6s 333ms/step - loss: 0.2531 - accuracy: 0.9691 - precision: 0.9776 - recall: 0.9284 - auc: 0.9952 - val_loss: 0.8168 - val_accuracy: 0.8140 - val_precision: 0.7355 - val_recall: 0.6899 - val_auc: 0.8778 - lr: 0.0010
Epoch 4/10
17/17 [==============================] - 5s 323ms/step - loss: 0.1725 - accuracy: 0.9884 - precision: 0.9902 - recall: 0.9749 - auc: 0.9993 - val_loss: 0.7385 - val_accuracy: 0.8217 - val_preci

In [9]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and resize the test image to match the model's input shape
test_image = image.load_img('../updatedImageSet2/angry/3880250382_260e6466c0_b.jpg', target_size=(224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Normalize the image
test_image /= 255.0


In [10]:

# Make a prediction using the model
result = model.predict(test_image)


1/1 [==============================] - 1s 516ms/step


In [11]:
class_indices = training_set.class_indices
class_labels = {v: k for k, v in class_indices.items()}

# Get the predicted class label
predicted_class_index = np.argmax(result)
predicted_class_label = class_labels[predicted_class_index]
predicted_class_label

'angry'

In [12]:
model.save("../saved_models/finalImageModel.h5")